# Spatio Temporal Files using extracted locations from the GDELT Knowledge Graph
We are reading the CSV files into a pandas dataframe.
The pandas dataframes are converted into a netcdf file using latitude, longitude and time.
We are using xarray for the conversion from dataframe to netcdf.

In [13]:
!pip install --user netcdf4
!pip install --user xarray

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
from datetime import date
import os
import pandas
import tempfile
import xarray

C:\bin\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text))


# Read the extracted locations from the temp folder into a dataframe

In [2]:
def read_gkg_locations_from_temp():
    gkg_locations = None
    with os.scandir(tempfile.gettempdir()) as dir_scanner:
        for dir_entry in dir_scanner:
            if dir_entry.is_file():
                if dir_entry.name.endswith(".gkg.csv"):
                        gkg_locations_temp = pandas.read_csv(dir_entry.path)
                        if gkg_locations:
                             pandas.concat([gkg_locations, gkg_locations_temp], axis=0)
                        else:
                            gkg_locations = gkg_locations_temp
    return gkg_locations

In [3]:
corona_locations = read_gkg_locations_from_temp()
corona_locations.head()

,GKGRECORDID,DATE,SourceCollectionIdentifier,SourceCommonName,DocumentIdentifier,Location_Name,Location_Lat,Location_Lon
0,20200213000000-23,20200213000000,1,myanmarnews.net,https://www.myanmarnews.net/news/264003252/cor...,"Tokyo, Tokyo, Japan",35.6850,139.75100
1,20200213000000-23,20200213000000,1,myanmarnews.net,https://www.myanmarnews.net/news/264003252/cor...,"Wuhan, Hubei, China",30.5833,114.26700
2,20200213000000-46,20200213000000,1,northkoreatimes.com,https://www.northkoreatimes.com/news/264003252...,"Tokyo, Tokyo, Japan",35.6850,139.75100
3,20200213000000-46,20200213000000,1,northkoreatimes.com,https://www.northkoreatimes.com/news/264003252...,"Wuhan, Hubei, China",30.5833,114.26700
4,20200213000000-51,20200213000000,1,pittsburghstar.com,https://www.pittsburghstar.com/news/264006117/...,"Barcelona, Comunidad Autonoma De Cataluna, Spain",41.3833,2.18333


# Convert the dataframe to a multidimensional xarray
- Set the dataframes index using latitude, longitude and date
- Drop the duplicate multi-index entries

In [4]:
corona_locations_multi = corona_locations.head().set_index(["Location_Lat", "Location_Lon", "DATE"])
corona_locations_multi = corona_locations_multi.loc[~corona_locations_multi.index.duplicated(keep="first")]
corona_locations_xarray = corona_locations_multi.to_xarray()

# Save the xarray as a netcdf file

In [6]:
corona_locations_xarray.to_netcdf("{}/corona_locations_{}.gkg.nc".format(tempfile.gettempdir(), date.today().strftime("%Y%m%d"), compute=True))

del corona_locations
del corona_locations_multi
del corona_locations_xarray

AttributeError: module 'dask.base' has no attribute 'get_scheduler'